# Retrieval-Augmented Shopping Assistant - EDA

This notebook explores the ABO dataset for initial insights.

In [2]:
import pandas as pd

In [3]:
# Load ABO image dataset
df_img = pd.read_csv("/kaggle/input/amazon-berkeley-objects/images/metadata/images.csv")  # or csv if applicable

In [38]:
# Preview
print("Shape:", df.shape)
print("Columns:", df.columns.tolist())
print("\nData Types:\n", df.dtypes)
print("\nMissing Values:\n", df.isnull().sum())

Shape: (398212, 4)
Columns: ['image_id', 'height', 'width', 'path']

Data Types:
 image_id    object
height       int64
width        int64
path        object
dtype: object

Missing Values:
 image_id    0
height      0
width       0
path        0
dtype: int64


In [15]:
df.loc[df['image_id'] == "81iZlv3bjpL"]

,image_id,height,width,path
319529,81iZlv3bjpL,2560,1969,8c/8ccb5859.jpg


In [19]:
# Load ABO image dataset
df_metadata = pd.read_json("/kaggle/input/listing/listings/metadata/listings_0.json", lines = True)  # or csv if applicable

In [39]:
print("Shape:", df_metadata.shape)
print("Columns:", df_metadata.columns.tolist())
print("\nData Types:\n", df_metadata.dtypes)
print("\nMissing Values:\n", df_metadata.isnull().sum())

Shape: (9232, 28)
Columns: ['brand', 'bullet_point', 'color', 'item_id', 'item_name', 'model_name', 'model_number', 'model_year', 'product_type', 'style', 'main_image_id', 'other_image_id', 'item_keywords', 'country', 'marketplace', 'domain_name', 'node', 'item_dimensions', 'item_weight', 'material', 'fabric_type', 'color_code', 'product_description', 'spin_id', '3dmodel_id', 'pattern', 'finish_type', 'item_shape']

Data Types:
 brand                  object
bullet_point           object
color                  object
item_id                object
item_name              object
model_name             object
model_number           object
model_year             object
product_type           object
style                  object
main_image_id          object
other_image_id         object
item_keywords          object
country                object
marketplace            object
domain_name            object
node                   object
item_dimensions        object
item_weight            obje

In [34]:
df_metadata['product_type'].value_counts()

product_type
[{'value': 'CELLULAR_PHONE_CASE'}]    4103
[{'value': 'SHOES'}]                   762
[{'value': 'GROCERY'}]                 412
[{'value': 'HOME'}]                    343
[{'value': 'HOME_BED_AND_BATH'}]       202
                                      ... 
[{'value': 'PLIERS'}]                    1
[{'value': 'TOYS_AND_GAMES'}]            1
[{'value': 'DEHUMIDIFIER'}]              1
[{'value': 'DRINK_FLAVORED'}]            1
[{'value': 'WHEEL_CUTTER'}]              1
Name: count, Length: 335, dtype: int64

In [37]:
df_metadata.isnull().sum

<bound method DataFrame.sum of       brand  bullet_point  color  item_id  item_name  model_name  \
0     False         False  False    False      False       False   
1     False         False  False    False      False        True   
2     False         False  False    False      False        True   
3     False          True  False    False      False        True   
4     False         False  False    False      False        True   
...     ...           ...    ...      ...        ...         ...   
9227  False         False  False    False      False       False   
9228  False         False  False    False      False       False   
9229  False          True   True    False      False       False   
9230  False         False  False    False      False       False   
9231  False         False  False    False      False        True   

      model_number  model_year  product_type  style  ...  item_weight  \
0            False       False         False  False  ...         True   
1     